In [1]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [2]:
# Leitura dos dados
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Correlaçãp entre as features e a feature "IN_TREINO"
numeric_features = train.select_dtypes(include=[np.number])
corr = numeric_features.corr()
print (corr['IN_TREINEIRO'].sort_values(ascending=False)[:10], '\n')

IN_TREINEIRO       1.000000
TP_ST_CONCLUSAO    0.533983
TP_PRESENCA_CN     0.094692
TP_PRESENCA_CH     0.094692
TP_PRESENCA_LC     0.092454
TP_PRESENCA_MT     0.092454
Q034               0.055265
Q039               0.040348
Q005               0.033411
Q038               0.018855
Name: IN_TREINEIRO, dtype: float64 



In [4]:
features = ['TP_ST_CONCLUSAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
                 'TP_PRESENCA_MT']
features_corr = ['IN_TREINEIRO', 'TP_ST_CONCLUSAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
                 'TP_PRESENCA_MT']
df_train = train[features_corr]
df_test = test[features]

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   IN_TREINEIRO     13730 non-null  int64
 1   TP_ST_CONCLUSAO  13730 non-null  int64
 2   TP_PRESENCA_CN   13730 non-null  int64
 3   TP_PRESENCA_CH   13730 non-null  int64
 4   TP_PRESENCA_LC   13730 non-null  int64
 5   TP_PRESENCA_MT   13730 non-null  int64
dtypes: int64(6)
memory usage: 643.7 KB


In [6]:
df_train.isnull().sum()

IN_TREINEIRO       0
TP_ST_CONCLUSAO    0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
TP_PRESENCA_MT     0
dtype: int64

In [7]:
df_test.isnull().sum()

TP_ST_CONCLUSAO    0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
TP_PRESENCA_MT     0
dtype: int64

In [8]:
y_train = df_train['IN_TREINEIRO']
df_train.drop('IN_TREINEIRO', axis=1, inplace=True)
x_train = df_train
x_test = df_test[features]
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)

C:\Users\GEA\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor( 
           criterion='mae', 
           max_depth=8,
           max_leaf_nodes=None,
           min_impurity_split=None,
           min_samples_leaf=1,
           min_samples_split=2,
           min_weight_fraction_leaf=0.0,
           n_estimators= 500,
           n_jobs=-1,
           random_state=0,
           verbose=0,
           warm_start=False
)

In [10]:
regressor.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [12]:
x_test = df_test[features] 
x_test = sc.transform(x_test)

In [13]:
y_pred_test = regressor.predict(x_test)
y_pred_train = regressor.predict(x_train)

In [14]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train).round(8)  )
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train).round(8) )  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)).round(8))

MAE: 0.03812673
MSE: 0.03804982
RMSE: 0.19506363


In [15]:
y_pred_test

array([0., 0., 1., ..., 0., 0., 0.])

In [16]:
type(y_pred_test)

numpy.ndarray

In [17]:
submission = pd.DataFrame()
submission['NU_INSCRICAO'] = test.NU_INSCRICAO
submission['IN_TREINEIRO'] = y_pred_test.astype(int)

In [18]:
submission.sample(20)

,NU_INSCRICAO,IN_TREINEIRO
201,a59bade0274b50feb84980cd6123d64d0b4bdebe,0
3830,4e249125fa47967ba3844370b6596e8f11e413e4,1
3291,a30c6b7911e556c1ae9d97eb125aad02a097b2e0,0
1154,9091e2edfb288c92a1737683f70933f52ed3549a,0
1997,dc71a4c16ad3566efee477ab928e8c9b09b7b532,0
2129,301877c849fac3f4ca16a0d9dd4e7442bc2e59f3,0
803,bcb2d2da35ded31809f6c9d1ba12b6ee3fd7b686,0
183,668624675573edf1139ae354c13623cf925ae92a,0
398,c13787760d72f755223f60f7935c694962f66893,0
4121,ee660e5adda8426f16132fa881acfc3a160132f5,0


In [19]:
submission.to_csv('answer.csv', index=False, header=True)